# **FEATURE IMPORTANCE**


First need to get the database from PostGRES

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
# Load in train.csv from S3 into a DataFrame
url = "https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/train_converted_df.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("train_converted_df.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

#Convert to Pandas to allow train_test_split usage
df = df.select("*").toPandas()
df = df.dropna(how="any")

+---+----------+-----------+-------+-----------+-----------+---------+------------+----------+---------+---------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+---------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+-----------+------------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-------+------+------+---------+-----------+-----------+-----------+-----------+-----------+----------+------------+------------+------------+---------------+---------------+---------------+----------------+-----------------+-------------+-------------+----------------+-------------+-------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------

Forest of Trees Feature Selection


In [23]:
#Get the Feature names
import pandas as pd

features = [col for col in df.columns]
print(features)

print(type(features))

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Pave', 'Alley_Pave', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_NoSeWa', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig

In [5]:
#Import sklearn tools and dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [6]:
#####################Create the feature data and the target

#Feature DataFrame is X
X = df.drop(columns=["Id", "SalePrice"])


#Target Dataframe is y
y = df["SalePrice"]


#####################Create train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [7]:
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [8]:
importances = rf.feature_importances_
importances

array([0.01930556, 0.02073413, 0.01548942, 0.01406746, 0.01662698,
       0.03017857, 0.02278439, 0.01467593, 0.01293651, 0.005     ,
       0.01240741, 0.        , 0.01425926, 0.02152778, 0.01809524,
       0.01285053, 0.02117063, 0.0144246 , 0.01416667, 0.01460317,
       0.015     , 0.01065476, 0.00555556, 0.01796296, 0.        ,
       0.00678571, 0.03005291, 0.01483466, 0.02680556, 0.        ,
       0.01141534, 0.01662037, 0.        , 0.01560185, 0.01426587,
       0.03390873, 0.02335317, 0.01049603, 0.01402116, 0.        ,
       0.        , 0.02511905, 0.00662037, 0.01224206, 0.        ,
       0.00402778, 0.02123016, 0.01706349, 0.00833333, 0.00449074,
       0.01785053, 0.0081746 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01409392, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01347222, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [24]:
features.remove("Id")
features.remove("SalePrice")

sorted(zip(rf.feature_importances_, features), reverse=True)

[(0.03390873015873015, 'GarageYrBlt'),
 (0.030178571428571423, 'YearBuilt'),
 (0.030052910052910057, 'FullBath'),
 (0.026805555555555555, 'BedroomAbvGr'),
 (0.025119047619047617, 'WoodDeckSF'),
 (0.0233531746031746, 'GarageFinish'),
 (0.02289021164021164, 'MasVnrType_None'),
 (0.02278439153439154, 'YearRemodAdd'),
 (0.02152777777777777, 'BsmtFinType1'),
 (0.02144179894179894, 'MasVnrType_BrkFace'),
 (0.02126984126984127, 'Foundation_CBlock'),
 (0.021230158730158724, 'PoolArea'),
 (0.02117063492063492, 'BsmtFinSF2'),
 (0.02073412698412698, 'LotFrontage'),
 (0.019305555555555555, 'MSSubClass'),
 (0.018095238095238095, 'BsmtFinSF1'),
 (0.017962962962962962, 'GrLivArea'),
 (0.0178505291005291, 'MoSold'),
 (0.017063492063492062, 'PoolQC'),
 (0.016944444444444443, 'Neighborhood_NWAmes'),
 (0.016626984126984126, 'OverallCond'),
 (0.01662037037037037, 'TotRmsAbvGrd'),
 (0.016547619047619044, 'RoofStyle_Gable'),
 (0.01621031746031746, 'Exterior2nd_Stucco'),
 (0.01560185185185185, 'Fireplaces'),